# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686565


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:36,  1.31s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:44,  1.63s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:40,  1.54s/it]

Rendering models:  20%|██        | 6/30 [00:06<00:26,  1.11s/it]

...

Rendering models:  80%|████████  | 24/30 [00:07<00:00,  7.05it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  8.47it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.35it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  9.48it/s]

not-logged-in-7e36d1f05e1fdd4c2b64    0.000106
kjkavene                              0.000314
mtajnai                               0.001427
bldelacr                              0.000168
justsomenoodles                       0.000126
jmfranci                              0.000533
mbaker23                              0.003526
not-logged-in-ef704c739eb8412f355e    0.000175
msalin03                              0.000577
Jboy2000                              0.045091
not-logged-in-3a93234a2311f6df1afd    0.089764
not-logged-in-9a45911eb71184fe7c7f    0.000427
not-logged-in-4c7a90b2649bf349f121    0.001502
acapirala                             0.000117
TRISTANM18                            0.000229
Lavadude                              0.002452
not-logged-in-6ea7dd94a335e5238780    0.000333
w7250369                              0.000489
Biloute                               0.000127
not-logged-in-f8c2f88355fba4282271    0.009256
not-logged-in-340a28051a320c10499e    0.021260
cheez__it    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())